# Exploratory Data Analysis Project
___

You will be working with the `covid19-can.csv` file located in the `Data` folder.

This dataset is obtained from the [Government of Canada Public Healht Infobase](https://www.activestate.com/resources/quick-reads/what-is-pandas-in-python-everything-you-need-to-know/) and contains information on daily reported COVID-19 cases as well as total COVID-19 deaths in all provinces of Canada.

**Analyze the above dataset to answer the following questions:**

1. What is the total number of COVID-19 cases reported in each province?
2. Which province has the highest average rate of COVID-19 per capita?
3. What is the average rate of COVID-19 deaths per capita?
4. What is the overall mortality rate of COVID-19 in Canada?
5. What is the mortality rate per province?
6. What are the total reported cases per year?
7. For each year in the dataset, find the month with the highest total number of cases.
8. For each year, find the month with the lowest total number of cases.
9. Which year had the highest mortality rate?
10. Which year had the lowest total number of cases?
11. Which year had the highest total number of cases?
12. In 2020, on which day did Quebex have the highest number of COVID-19 deaths?

## Step 1: Imports

In [1]:
import pandas as pd
import numpy as np

## Step 2: Reading Data

In [4]:
df = pd.read_csv('covid19-download.csv')
df
df.head()

,pruid,prname,prnameFR,date,reporting_week,reporting_year,update,totalcases,numtotal_last7,ratecases_total,...,ratecases_last7,ratedeaths_last7,numtotal_last14,numdeaths_last14,ratetotal_last14,ratedeaths_last14,avgcases_last7,avgincidence_last7,avgdeaths_last7,avgratedeaths_last7
0,59,British Columbia,Colombie-Britannique,2020-02-08,6,2020,1.0,4,3,0.07,...,0.05,0.0,4,0.0,0.07,0.0,0.43,0.01,0.0,0.0
1,48,Alberta,Alberta,2020-02-08,6,2020,1.0,0,0,0,...,0,0.0,0,0.0,0,0.0,0,0,0.0,0.0
2,47,Saskatchewan,Saskatchewan,2020-02-08,6,2020,1.0,0,0,0,...,0,0.0,0,0.0,0,0.0,0,0,0.0,0.0
3,46,Manitoba,Manitoba,2020-02-08,6,2020,1.0,0,0,0,...,0,0.0,0,0.0,0,0.0,0,0,0.0,0.0
4,35,Ontario,Ontario,2020-02-08,6,2020,1.0,4,1,0.03,...,0.01,0.0,1,0.0,0.01,0.0,0.14,0,0.0,0.0


In [6]:
df.tail()

,pruid,prname,prnameFR,date,reporting_week,reporting_year,update,totalcases,numtotal_last7,ratecases_total,...,ratecases_last7,ratedeaths_last7,numtotal_last14,numdeaths_last14,ratetotal_last14,ratedeaths_last14,avgcases_last7,avgincidence_last7,avgdeaths_last7,avgratedeaths_last7
3610,60,Yukon,Yukon,2024-09-14,37,2024,0.0,-,-,-,...,-,NaN,-,NaN,-,NaN,-,-,NaN,NaN
3611,61,Northwest Territories,Territoires du Nord-Ouest,2024-09-14,37,2024,0.0,-,-,-,...,-,NaN,-,NaN,-,NaN,-,-,NaN,NaN
3612,62,Nunavut,Nunavut,2024-09-14,37,2024,0.0,-,-,-,...,-,NaN,-,NaN,-,NaN,-,-,NaN,NaN
3613,99,Repatriated travellers,Voyageurs rapatriés,2024-09-14,37,2024,NaN,-,-,-,...,-,NaN,-,0.0,-,NaN,-,-,0.00,NaN
3614,1,Canada,Canada,2024-09-14,37,2024,NaN,-,-,-,...,-,0.11,-,89.0,-,0.22,-,-,6.29,0.02


## Step 3: Data Exploration

Explore the dataset to better understand its characteristics, structure, content and data types.

In [9]:
display(df.describe())

,pruid,reporting_week,reporting_year,update,numdeaths,numdeaths_last7,ratedeaths,ratedeaths_last7,numdeaths_last14,ratedeaths_last14,avgdeaths_last7,avgratedeaths_last7
count,3615.000000,3615.000000,3615.000000,3133.000000,3615.000000,3082.000000,3374.000000,2841.000000,3099.000000,2858.000000,3082.000000,2841.000000
mean,39.200000,25.946058,2021.908714,0.858602,4895.237621,39.296561,65.562587,0.568244,78.138432,1.127446,5.613605,0.081130
std,26.287218,14.550301,1.354436,0.348487,10921.498719,108.495337,58.125032,0.888010,214.736758,1.626828,15.499336,0.126037
min,1.000000,1.000000,2020.000000,0.000000,0.000000,-1.000000,0.000000,-0.080000,-1.000000,-0.080000,-0.140000,-0.010000
25%,12.000000,14.000000,2021.000000,1.000000,7.000000,0.000000,9.830000,0.000000,0.000000,0.020000,0.000000,0.000000
50%,46.000000,26.000000,2022.000000,1.000000,345.000000,4.000000,56.935000,0.270000,9.000000,0.580000,0.570000,0.040000
75%,60.000000,38.000000,2023.000000,1.000000,3805.500000,29.000000,109.262500,0.740000,58.000000,1.530000,4.140000,0.110000
max,99.000000,53.000000,2024.000000,1.000000,60769.000000,1317.000000,231.040000,11.260000,2595.000000,17.880000,188.140000,1.610000


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3615 entries, 0 to 3614
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pruid                3615 non-null   int64  
 1   prname               3615 non-null   object 
 2   prnameFR             3615 non-null   object 
 3   date                 3615 non-null   object 
 4   reporting_week       3615 non-null   int64  
 5   reporting_year       3615 non-null   int64  
 6   update               3133 non-null   float64
 7   totalcases           3615 non-null   object 
 8   numtotal_last7       3236 non-null   object 
 9   ratecases_total      3390 non-null   object 
 10  numdeaths            3615 non-null   int64  
 11  numdeaths_last7      3082 non-null   float64
 12  ratedeaths           3374 non-null   float64
 13  ratecases_last7      3011 non-null   object 
 14  ratedeaths_last7     2841 non-null   float64
 15  numtotal_last14      3242 non-null   o

## Step 4: Data Preparation

In [14]:
# Check for missing values
df.isnull().sum()

pruid                    0
prname                   0
prnameFR                 0
date                     0
reporting_week           0
reporting_year           0
update                 482
totalcases               0
numtotal_last7         379
ratecases_total        225
numdeaths                0
numdeaths_last7        533
ratedeaths             241
ratecases_last7        604
ratedeaths_last7       774
numtotal_last14        373
numdeaths_last14       516
ratetotal_last14       598
ratedeaths_last14      757
avgcases_last7         379
avgincidence_last7     604
avgdeaths_last7        533
avgratedeaths_last7    774
dtype: int64

In [16]:
# Find all unique values in the 'prname' column
unique_prnames = df['prname'].unique()
unique_prnames

array(['British Columbia', 'Alberta', 'Saskatchewan', 'Manitoba',
       'Ontario', 'Quebec', 'Newfoundland and Labrador', 'New Brunswick',
       'Nova Scotia', 'Prince Edward Island', 'Yukon',
       'Northwest Territories', 'Nunavut', 'Repatriated travellers',
       'Canada'], dtype=object)

In [18]:
# Convert object columns to numeric where necessary
cols_to_convert = ['totalcases', 'numtotal_last7', 'ratecases_total', 'ratecases_last7']  # Specify numeric columns
for col in cols_to_convert:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Convert 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Create 'year' and 'month' columns
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

# Fill missing values for numeric columns with the mean of each column
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

# Fill missing values for categorical columns with the most frequent value (mode)
categorical_cols = df.select_dtypes(include='object').columns
for col in categorical_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)

# Check if all missing values have been handled
print(df.isnull().sum())

pruid                  0
prname                 0
prnameFR               0
date                   0
reporting_week         0
reporting_year         0
update                 0
totalcases             0
numtotal_last7         0
ratecases_total        0
numdeaths              0
numdeaths_last7        0
ratedeaths             0
ratecases_last7        0
ratedeaths_last7       0
numtotal_last14        0
numdeaths_last14       0
ratetotal_last14       0
ratedeaths_last14      0
avgcases_last7         0
avgincidence_last7     0
avgdeaths_last7        0
avgratedeaths_last7    0
year                   0
month                  0
dtype: int64


In [28]:
# Display the number of rows before deletion
print(f"Number of rows before deletion: {len(df)}")

# Delete all records with 'Canada' in the 'prname' column
df = df[df['prname'] != 'Canada']

# Display the number of rows after deletion
print(f"Number of rows after deletion: {len(df)}")

# Display the first few rows to verify changes
print(df.head())

Number of rows before deletion: 3374
Number of rows after deletion: 3374
   pruid            prname              prnameFR       date  reporting_week  \
0     59  British Columbia  Colombie-Britannique 2020-02-08               6   
1     48           Alberta               Alberta 2020-02-08               6   
2     47      Saskatchewan          Saskatchewan 2020-02-08               6   
3     46          Manitoba              Manitoba 2020-02-08               6   
4     35           Ontario               Ontario 2020-02-08               6   

   reporting_year  update  totalcases  numtotal_last7  ratecases_total  ...  \
0            2020     1.0         4.0             3.0             0.07  ...   
1            2020     1.0         0.0             0.0             0.00  ...   
2            2020     1.0         0.0             0.0             0.00  ...   
3            2020     1.0         0.0             0.0             0.00  ...   
4            2020     1.0         4.0             1.0    

## Step 5: Data Analysis

In [30]:
# 1. Total number of COVID-19 cases reported in each province
total_cases_per_province = df.groupby('prname')['totalcases'].sum().reset_index()
total_cases_per_province.columns = ['Province', 'Total Cases']

print(total_cases_per_province)

                     Province   Total Cases
0                     Alberta  9.561097e+07
1            British Columbia  6.308603e+07
2                    Manitoba  2.770823e+07
3               New Brunswick  1.620435e+07
4   Newfoundland and Labrador  1.240875e+07
5       Northwest Territories  7.379367e+06
6                 Nova Scotia  2.203921e+07
7                     Nunavut  6.419299e+06
8                     Ontario  2.235674e+08
9        Prince Edward Island  1.216706e+07
10                     Quebec  1.856742e+08
11     Repatriated travellers  5.964904e+06
12               Saskatchewan  2.747007e+07
13                      Yukon  6.579431e+06


In [38]:
# 2. Check the distribution of 'ratecases_total'
print(df['ratecases_total'].describe())

# Drop rows where 'ratecases_total' is NaN
df = df.dropna(subset=['ratecases_total'])

# Calculate the average rate of COVID-19 cases per capita for each province
average_rate_per_province = df.groupby('prname')['ratecases_total'].mean().dropna()

# Check if there are still values available after dropping NaNs
print(average_rate_per_province)

count     3374.000000
mean      7786.536902
std       7286.202812
min          0.000000
25%        828.372500
50%       7735.473470
75%      10949.637500
max      33715.410000
Name: ratecases_total, dtype: float64
prname
Alberta                       8436.119110
British Columbia              4808.329317
Manitoba                      6715.564172
New Brunswick                 5605.162347
Newfoundland and Labrador     5480.048446
Northwest Territories        13590.503052
Nova Scotia                   6814.735957
Nunavut                       5178.267367
Ontario                       6298.446330
Prince Edward Island         15328.763882
Quebec                        8916.040894
Repatriated travellers        7735.473470
Saskatchewan                  7894.614546
Yukon                         6209.447741
Name: ratecases_total, dtype: float64


In [40]:
# 2. Province with the highest average rate of COVID-19 cases per capita
if not df['ratecases_total'].isnull().all():
    # Calculate the average rate of COVID-19 cases per capita for each province
    average_rate_per_province = df.groupby('prname')['ratecases_total'].mean()

    # Drop NaN values from the resulting series (though ideally there should be none left)
    average_rate_per_province = average_rate_per_province.dropna()

    # Find the province with the highest average rate of COVID-19 cases per capita
    if not average_rate_per_province.empty:
        highest_average_rate_province = average_rate_per_province.idxmax()
        highest_average_rate_value = average_rate_per_province.max()
        print(f"{highest_average_rate_province} had the highest average rate of {highest_average_rate_value:.2f} cases per capita.")
    else:
        print("No valid data available for calculating the highest average rate.")
else:
    print("No data available for 'ratecases_total' to calculate average rate per province.")

Prince Edward Island had the highest average rate of 15328.76 cases per capita.


In [42]:
# 3. Average rate of COVID-19 deaths per capita in each province
average_death_rate_per_province = df.groupby('prname')['ratedeaths'].mean()
print("\nAverage rate of COVID-19 deaths per capita in each province:\n", average_death_rate_per_province)


Average rate of COVID-19 deaths per capita in each province:
 prname
Alberta                       80.861494
British Columbia              64.224232
Manitoba                     110.904066
New Brunswick                 55.342407
Newfoundland and Labrador     33.408133
Northwest Territories         29.442282
Nova Scotia                   44.100249
Nunavut                       11.659585
Ontario                       76.513237
Prince Edward Island          29.014315
Quebec                       151.454606
Repatriated travellers        65.562587
Saskatchewan                  97.587510
Yukon                         41.802241
Name: ratedeaths, dtype: float64


In [44]:
# 4. Overall mortality rate of COVID-19 in Canada
total_cases = df['totalcases'].sum()
total_deaths = df['numdeaths'].sum()

# Calculate the overall mortality rate
if total_cases > 0:
    overall_mortality_rate = (total_deaths / total_cases) * 100
    print("\nThe overall mortality rate of COVID-19 in Canada is {:.2f}%.".format(overall_mortality_rate))
else:
    print("No total cases available to calculate overall mortality rate.")


The overall mortality rate of COVID-19 in Canada is 1.24%.


In [46]:
# 5. Mortality rate per province
# Calculate total cases and total deaths per province
province_totals = df.groupby('prname')[['totalcases', 'numdeaths']].sum()

# Calculate the mortality rate per province
province_totals['mortality_rate'] = (province_totals['numdeaths'] / province_totals['totalcases']) * 100
print("\nMortality rate per province:\n", province_totals)


Mortality rate per province:
                              totalcases  numdeaths  mortality_rate
prname                                                            
Alberta                    9.561097e+07     915000        0.957003
British Columbia           6.308603e+07     854239        1.354086
Manitoba                   2.770823e+07     388866        1.403431
New Brunswick              1.620435e+07     111327        0.687019
Newfoundland and Labrador  1.240875e+07      43365        0.349471
Northwest Territories      7.379367e+06       3191        0.043242
Nova Scotia                2.203921e+07     112519        0.510540
Nunavut                    6.419299e+06       1143        0.017806
Ontario                    2.235674e+08    2878127        1.287364
Prince Edward Island       1.216706e+07      12152        0.099876
Quebec                     1.856742e+08    3239317        1.744624
Repatriated travellers     5.964904e+06          0        0.000000
Saskatchewan               2.74

In [48]:
# 6. Total reported cases per year
total_cases_per_year = df.groupby('year')['totalcases'].sum()
print("\nTotal reported cases per year:\n", total_cases_per_year)


Total reported cases per year:
 year
2020    6.505795e+06
2021    6.910618e+07
2022    2.049785e+08
2023    2.444746e+08
2024    1.872143e+08
Name: totalcases, dtype: float64


In [50]:
# 7. Month with the highest total cases for each year
# Calculate total cases per year and month
cases_per_year_month = df.groupby(['year', 'month'])['totalcases'].sum().reset_index()

# Find the month with the highest total cases for each year
max_cases_per_year = cases_per_year_month.loc[cases_per_year_month.groupby('year')['totalcases'].idxmax()]
print("\nMonth with the highest total cases for each year:\n", max_cases_per_year)


Month with the highest total cases for each year:
     year  month    totalcases
10  2020     12  1.916862e+06
20  2021     10  8.371906e+06
34  2022     12  2.238220e+07
46  2023     12  2.434284e+07
52  2024      6  2.608399e+07


In [52]:
# 8. Month with the lowest total cases for each year
min_cases_per_year = cases_per_year_month.loc[cases_per_year_month.groupby('year')['totalcases'].idxmin()]
print("\nMonth with the lowest total cases for each year:\n", min_cases_per_year)


Month with the lowest total cases for each year:
     year  month    totalcases
0   2020      2  5.200000e+01
12  2021      2  3.315944e+06
24  2022      2  1.292752e+07
35  2023      1  1.818144e+07
55  2024      9  1.043360e+07


In [54]:
# 9. Yearly Mortality Rate
# Calculate total deaths and total cases per year, then merge the results
yearly_data = df.groupby('year').agg(total_deaths=('numdeaths', 'sum'),
                                     total_cases=('totalcases', 'sum')).reset_index()

# Calculate the mortality rate per year
yearly_data['mortality_rate'] = (yearly_data['total_deaths'] / yearly_data['total_cases']) * 100

# Print the yearly mortality rate
print("\nYearly mortality rate:\n", yearly_data[['year', 'mortality_rate']])


Yearly mortality rate:
    year  mortality_rate
0  2020        5.117038
1  2021        1.875161
2  2022        1.077301
3  2023        1.148952
4  2024        1.176329


In [56]:
# 10. Year with Lowest Total Cases
# Find the year with the lowest total number of cases
if not yearly_data['total_cases'].isnull().all():
    year_with_lowest_cases = yearly_data.loc[yearly_data['total_cases'].idxmin()]
    print("\nYear with the lowest total cases:\n", year_with_lowest_cases)
else:
    print("\nNo valid data available to determine the year with the lowest total cases.")


Year with the lowest total cases:
 year              2.020000e+03
total_deaths      3.329040e+05
total_cases       6.505795e+06
mortality_rate    5.117038e+00
Name: 0, dtype: float64


In [58]:
# 11. Year with Highest Total Cases
# Find the year with the highest total number of cases
if not yearly_data['total_cases'].isnull().all():
    year_with_highest_cases = yearly_data.loc[yearly_data['total_cases'].idxmax()]
    print("\nYear with the highest total cases:\n", year_with_highest_cases)
else:
    print("\nNo valid data available to determine the year with the highest total cases.")


Year with the highest total cases:
 year              2.023000e+03
total_deaths      2.808896e+06
total_cases       2.444746e+08
mortality_rate    1.148952e+00
Name: 3, dtype: float64


In [60]:
# 12. Day with Highest Number of COVID-19 Deaths for Quebec in 2020
# Filter the dataset for Quebec in 2020
quebec_2020 = df[(df['prname'] == 'Quebec') & (df['year'] == 2020)]

# Check if Quebec 2020 data is not empty
if not quebec_2020.empty:
    # Find the day with the highest number of COVID-19 deaths
    day_with_highest_deaths = quebec_2020.loc[quebec_2020['numdeaths'].idxmax()]
    print("\nDay with the highest number of COVID-19 deaths for Quebec in 2020:\n", day_with_highest_deaths)
else:
    print("\nNo data available for Quebec in 2020 to determine the day with the highest number of deaths.")


Day with the highest number of COVID-19 deaths for Quebec in 2020:
 pruid                                   24
prname                              Quebec
prnameFR                            Québec
date                   2020-12-26 00:00:00
reporting_week                          52
reporting_year                        2020
update                                 1.0
totalcases                        186323.0
numtotal_last7                     15385.0
ratecases_total                    2099.49
numdeaths                             7662
numdeaths_last7                      314.0
ratedeaths                           86.34
ratecases_last7                     173.36
ratedeaths_last7                      3.54
numtotal_last14                      29339
numdeaths_last14                     562.0
ratetotal_last14                    330.59
ratedeaths_last14                     6.33
avgcases_last7                     2197.86
avgincidence_last7                   24.77
avgdeaths_last7             

The End!